In [6]:
import pandas as pd
from datetime import datetime
import re
#Load our CSV files
file_name = 'TV2_1-93.csv'
sport = 'sport.csv'
vejr= 'Data-Vejr.csv'
# Read the CSV file from the current working directory
df_1_93 = pd.read_csv(file_name)
df_sport = pd.read_csv(sport)
df_vejr = pd.read_csv(vejr)

boy = 'Name_Boy.xlsx'
girl = 'Name_Girl.xlsx'
df_boy = pd.read_excel(boy)
df_girl = pd.read_excel(girl)

In [7]:
#We merge our CSV files together
merged_df = pd.concat([df_1_93, df_sport, df_vejr], ignore_index=True)
merged_names = pd.concat([df_boy, df_girl], ignore_index = True)

In [8]:
#Format the time
def format_timestamp(timestamp):
    dt_object = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
    return dt_object.strftime("%Y-%m-%d %H:%M")

# Apply the function to the 'Time' column
merged_df['Time'] = merged_df['Time'].apply(format_timestamp)

In [9]:
#Add 2 hours to summertime, add 1 hour to winter time
merged_df['Time'] = pd.to_datetime(merged_df['Time'], format='%Y-%m-%d %H:%M')

# Custom function to adjust time based on conditions
def adjust_time(row):
    if 3 <= row['Time'].month <= 10:
        return row['Time'] + pd.Timedelta(hours=2)
    else:
        return row['Time'] + pd.Timedelta(hours=1)

# Apply the custom function to the 'Time' column
merged_df['Time'] = merged_df.apply(adjust_time, axis=1)

In [10]:
#Here we add the three variables for the time
merged_df['Time'] = pd.to_datetime(merged_df['Time'], format='%Y-%m-%d %H:%M:%S')

# Define a function to assign time group based on the time of day
def assign_time_group(time):
    if time.time() >= pd.Timestamp('00:01:00').time() and time.time() <= pd.Timestamp('08:00:00').time():
        return 'morning'
    elif time.time() >= pd.Timestamp('08:01:00').time() and time.time() <= pd.Timestamp('16:00:00').time():
        return 'day'
    else:
        return 'evening'

# Apply the function to create the 'time_group' column
merged_df['time_group'] = merged_df['Time'].apply(assign_time_group)


In [11]:
# Define a function to calculate LIX score
def calculate_lix(text):
    if "Her giver vi dig dagens nyheder i kort form. Vi opdaterer hele tiden, hele dagen. Du kan tippe TV 2s journalist på livecenter@tv2.dk" in text or "Denne liveblog opdateres ikke længere." in text:
        return None  # Exclude specific texts
    words = re.findall(r'\w+', text)  # Tokenize words
    sentences = text.count('.') + text.count('!') + text.count('?')  # Count sentences
    long_words = sum(1 for word in words if len(word) > 6)  # Count long words
    lix = (len(words) / sentences) + (long_words * 100) / len(words)
    return lix

# Calculate LIX scores and add to new column 'LIX' in merged_df
merged_df['LIX'] = merged_df['All_text'].apply(calculate_lix)

# Print the DataFrame with LIX scores
merged_df


,Category,Authors,Headline,Time,Live,All_text,time_group,LIX
0,Udland,Mathias Hobolth Østerlund,Efter to år med Taleban er kvindernes sidste f...,2023-08-15 10:38:00,0,['Efter to år ved magten har Taliban gjort det...,day,39.097430
1,Samfund,Signe Marie Frost,"Elever bruger to smuthuller i telefonforbud, f...",2023-08-15 10:38:00,0,['Eleverne vil altid være i stand til at finde...,day,39.229083
2,Lokalt,"Amanda Nygaard Frisk, Jessica Skovmose",Færre færgeafgange til øer presser landmænd,2023-08-15 09:56:00,0,"['På Fejø, Femø og Askø skal en ny elfærge fin...",day,38.546371
3,Samfund,Amalie Abildgaard,Tidligere islamist: - Man skal 100 procent tag...,2023-08-15 09:01:00,0,"['Al-Qaeda har udsendt en trussel mod Danmark,...",day,42.699095
4,Krigen i Ukraine,Livecenter,Seneste nyt om krigen i Ukraine,2023-08-15 09:00:00,1,['Hvad sker der? Rusland har udført et stort l...,day,44.711451
...,...,...,...,...,...,...,...,...
5746,Vejr,Jonas Damsbo,Ny kulderekord i Kina,2023-01-24 16:41:00,0,"['Kina har sat ny officiel kulderekord, efter ...",evening,37.546268
5747,Vejr,Jonas Damsbo,Pænt tirsdagsvejr – her kommer der mest sol,2023-01-24 07:30:00,0,['Temperaturen forventes at nå over frysepunkt...,morning,32.914980
5748,Vejr,Jonas Damsbo,Spektakulært isfænomen vokser frem i danske skove,2023-01-23 18:20:00,0,"['Fænomenet ligner fine, hvide hår og går unde...",evening,36.754386
5749,Vejr,Jeppe Lykke Hansen,Sæler forvirret af varmt vejr – kan ikke finde...,2023-01-23 18:20:00,0,['Når vejret er for varmt til det tykke lag vi...,evening,36.179444


In [20]:

merged_df['first_name'] = merged_df['Authors'].str.split().str[0]

merged_df['Boy'] = merged_df['first_name'].isin(df_boy['Navn']).astype(int)
merged_df['Girl'] = merged_df['first_name'].isin(df_girl['Navn']).astype(int)
merged_df

filtered_df = merged_df[(merged_df['Boy'] == 0) & (merged_df['Girl'] == 0)]
frequency_table = filtered_df['first_name'].value_counts()
#print(frequency_table)

merged_df['Nyhedsbureau'] = merged_df['first_name'].isin(frequency_table.index).astype(int)
merged_df





,Category,Authors,Headline,Time,Live,All_text,time_group,LIX,Boy,first_name,Girl,Nyhedsbureau
0,Udland,Mathias Hobolth Østerlund,Efter to år med Taleban er kvindernes sidste f...,2023-08-15 10:38:00,0,['Efter to år ved magten har Taliban gjort det...,day,39.097430,1,Mathias,0,0
1,Samfund,Signe Marie Frost,"Elever bruger to smuthuller i telefonforbud, f...",2023-08-15 10:38:00,0,['Eleverne vil altid være i stand til at finde...,day,39.229083,0,Signe,1,0
2,Lokalt,"Amanda Nygaard Frisk, Jessica Skovmose",Færre færgeafgange til øer presser landmænd,2023-08-15 09:56:00,0,"['På Fejø, Femø og Askø skal en ny elfærge fin...",day,38.546371,0,Amanda,1,0
3,Samfund,Amalie Abildgaard,Tidligere islamist: - Man skal 100 procent tag...,2023-08-15 09:01:00,0,"['Al-Qaeda har udsendt en trussel mod Danmark,...",day,42.699095,0,Amalie,1,0
4,Krigen i Ukraine,Livecenter,Seneste nyt om krigen i Ukraine,2023-08-15 09:00:00,1,['Hvad sker der? Rusland har udført et stort l...,day,44.711451,0,Livecenter,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5746,Vejr,Jonas Damsbo,Ny kulderekord i Kina,2023-01-24 16:41:00,0,"['Kina har sat ny officiel kulderekord, efter ...",evening,37.546268,1,Jonas,0,0
5747,Vejr,Jonas Damsbo,Pænt tirsdagsvejr – her kommer der mest sol,2023-01-24 07:30:00,0,['Temperaturen forventes at nå over frysepunkt...,morning,32.914980,1,Jonas,0,0
5748,Vejr,Jonas Damsbo,Spektakulært isfænomen vokser frem i danske skove,2023-01-23 18:20:00,0,"['Fænomenet ligner fine, hvide hår og går unde...",evening,36.754386,1,Jonas,0,0
5749,Vejr,Jeppe Lykke Hansen,Sæler forvirret af varmt vejr – kan ikke finde...,2023-01-23 18:20:00,0,['Når vejret er for varmt til det tykke lag vi...,evening,36.179444,1,Jeppe,0,0


In [29]:
merged_df['Category'].value_counts()


Samfund          1780
Udland           1554
Sport             637
Vejr              619
Krimi             414
Politik           384
Business          179
Other             124
Underholdning      60
Name: Category, dtype: int64

In [28]:
#rename the category variable

merged_df['Category'] = merged_df['Category'].replace(['Nyheder', 'Dagens overblik', 'Lokalt'], 'Samfund')
merged_df['Category'] = merged_df['Category'].replace(['Klima'], 'Politik')
merged_df['Category'] = merged_df['Category'].replace(["GO'"], 'Underholdning')
merged_df['Category'] = merged_df['Category'].replace(['Krigen i Ukraine'], 'Udland')
merged_df['Category'] = merged_df['Category'].replace(['Penge'], 'Business')
merged_df['Category'] = merged_df['Category'].replace(['Emilie Meng-sagen', 'Det grænseløse bedrag', 'Skyderi i Fields','Svindel med udbytteskat'], 'Krimi')
merged_df['Category'] = merged_df['Category'].replace(['Tour de France', 'Royal Run'], 'Sport')
merged_df['Category'] = merged_df['Category'].replace(['Styret af mad',
       'Hemmeligheden i skyggearkivet', 'Balladen om kolonihaven',
       'Jordskælv i Tyrkiet og Syrien', 'Gældskaos', 'Donald Trump',
        'Tech',  'Energikrise','Mareridtet på Sneppevej', 'Priserne stiger', 'Coronavirus',
       'Gaslæk ved Bornholm', 'Det sidste ord', 'Lippert', 'Inflation',
       'Farlige forældre', 'Kernen', 'TV 2 Dokumentar',
       'Tirsdagsanalysen', 'Spionerne bag skærmen',
       'Eurovision', 'Ombord', 'Som sunket i jorden',
       'En gang narkostrømer',
       'Bodil Jørgensen – Man ved aldrig med i morgen',
       'Kroning af kong Charles III', 'De usynlige hjemløse',
       'Herlufsholms hemmeligheder', 'Operation X', 'Håbløst arbejde',
       'Hold mig fast. Før jeg forsvinder', 'Sidney', 'Alle vores børn',
       'Pentagon-lækagen', 'Røgsløret fra tobaksindustrien', 'OK23'],'Other')


merged_df

,Category,Authors,Headline,Time,Live,All_text,time_group,LIX,Boy,first_name,Girl,Nyhedsbureau
0,Udland,Mathias Hobolth Østerlund,Efter to år med Taleban er kvindernes sidste f...,2023-08-15 10:38:00,0,['Efter to år ved magten har Taliban gjort det...,day,39.097430,1,Mathias,0,0
1,Samfund,Signe Marie Frost,"Elever bruger to smuthuller i telefonforbud, f...",2023-08-15 10:38:00,0,['Eleverne vil altid være i stand til at finde...,day,39.229083,0,Signe,1,0
2,Samfund,"Amanda Nygaard Frisk, Jessica Skovmose",Færre færgeafgange til øer presser landmænd,2023-08-15 09:56:00,0,"['På Fejø, Femø og Askø skal en ny elfærge fin...",day,38.546371,0,Amanda,1,0
3,Samfund,Amalie Abildgaard,Tidligere islamist: - Man skal 100 procent tag...,2023-08-15 09:01:00,0,"['Al-Qaeda har udsendt en trussel mod Danmark,...",day,42.699095,0,Amalie,1,0
4,Udland,Livecenter,Seneste nyt om krigen i Ukraine,2023-08-15 09:00:00,1,['Hvad sker der? Rusland har udført et stort l...,day,44.711451,0,Livecenter,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5746,Vejr,Jonas Damsbo,Ny kulderekord i Kina,2023-01-24 16:41:00,0,"['Kina har sat ny officiel kulderekord, efter ...",evening,37.546268,1,Jonas,0,0
5747,Vejr,Jonas Damsbo,Pænt tirsdagsvejr – her kommer der mest sol,2023-01-24 07:30:00,0,['Temperaturen forventes at nå over frysepunkt...,morning,32.914980,1,Jonas,0,0
5748,Vejr,Jonas Damsbo,Spektakulært isfænomen vokser frem i danske skove,2023-01-23 18:20:00,0,"['Fænomenet ligner fine, hvide hår og går unde...",evening,36.754386,1,Jonas,0,0
5749,Vejr,Jeppe Lykke Hansen,Sæler forvirret af varmt vejr – kan ikke finde...,2023-01-23 18:20:00,0,['Når vejret er for varmt til det tykke lag vi...,evening,36.179444,1,Jeppe,0,0


In [35]:

merged_df['Headline1'] = merged_df['Headline'].str.cat(['.'] * len(merged_df))
merged_df['LIX_Headline'] = merged_df['Headline1'].apply(calculate_lix)
merged_df.drop('Headline1', axis=1, inplace=True)
merged_df

,Category,Authors,Headline,Time,Live,All_text,time_group,LIX,Boy,first_name,Girl,Nyhedsbureau,LIX_Headline
0,Udland,Mathias Hobolth Østerlund,Efter to år med Taleban er kvindernes sidste f...,2023-08-15 10:38:00,0,['Efter to år ved magten har Taliban gjort det...,day,39.097430,1,Mathias,0,0,50.000000
1,Samfund,Signe Marie Frost,"Elever bruger to smuthuller i telefonforbud, f...",2023-08-15 10:38:00,0,['Eleverne vil altid være i stand til at finde...,day,39.229083,0,Signe,1,0,58.000000
2,Samfund,"Amanda Nygaard Frisk, Jessica Skovmose",Færre færgeafgange til øer presser landmænd,2023-08-15 09:56:00,0,"['På Fejø, Femø og Askø skal en ny elfærge fin...",day,38.546371,0,Amanda,1,0,56.000000
3,Samfund,Amalie Abildgaard,Tidligere islamist: - Man skal 100 procent tag...,2023-08-15 09:01:00,0,"['Al-Qaeda har udsendt en trussel mod Danmark,...",day,42.699095,0,Amalie,1,0,56.454545
4,Udland,Livecenter,Seneste nyt om krigen i Ukraine,2023-08-15 09:00:00,1,['Hvad sker der? Rusland har udført et stort l...,day,44.711451,0,Livecenter,0,1,39.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5746,Vejr,Jonas Damsbo,Ny kulderekord i Kina,2023-01-24 16:41:00,0,"['Kina har sat ny officiel kulderekord, efter ...",evening,37.546268,1,Jonas,0,0,29.000000
5747,Vejr,Jonas Damsbo,Pænt tirsdagsvejr – her kommer der mest sol,2023-01-24 07:30:00,0,['Temperaturen forventes at nå over frysepunkt...,morning,32.914980,1,Jonas,0,0,21.285714
5748,Vejr,Jonas Damsbo,Spektakulært isfænomen vokser frem i danske skove,2023-01-23 18:20:00,0,"['Fænomenet ligner fine, hvide hår og går unde...",evening,36.754386,1,Jonas,0,0,35.571429
5749,Vejr,Jeppe Lykke Hansen,Sæler forvirret af varmt vejr – kan ikke finde...,2023-01-23 18:20:00,0,['Når vejret er for varmt til det tykke lag vi...,evening,36.179444,1,Jeppe,0,0,28.666667
